In [39]:
import os
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import ast
import re
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter


In [19]:
summaries_war = pd.read_csv("../war_action_movies_with_persons.csv")
summaries_war.head()

,name,release_date,revenue,runtime,languages,country,genres,plot,Persona
0,Army of Darkness,1992.0,21502796.0,81.0,English,United States of America,"Cult, Horror, Stop motion, Costume drama, Acti...","After being pulled through a time portal, Ash ...","{'Williams': {'lands'}, 'Arthur': {'war'}, 'As..."
1,The Birth of a Nation,1915.0,50000000.0,190.0,"Silent film, English",United States of America,"Silent film, Indie, Costume drama, Epic, Black...",The film follows two juxtaposed families: the...,"{'Ben Cameron': {'guilty', 'leading', 'holds',..."
2,Barry Lyndon,1975.0,20000000.0,187.0,"French , English , German","United States of America, United Kingdom","Costume drama, Film adaptation, Period piece, ...",:By What Means Redmond Barry Acquired the Sty...,"{'Redmond Barry': {'decides', 'reveals', 'refu..."
3,Buffy the Vampire Slayer,1992.0,16624456.0,86.0,English,United States of America,"Action, Horror, Comedy, Horror Comedy",Buffy Summers is introduced as a stereotypica...,"{'Buffy Summers': {'confronts', 'Slayer', 'rea..."
4,Braveheart,1995.0,211409945.0,175.0,"French , Latin , English , Gaelic",United States of America,"Biography, Adventure, History, War film, Actio...","In the 13th century, after several years of po...","{'Longshanks': {'implores', 'seeks', 'sees', '..."


In [42]:
# Create an empty list to hold the new rows
new_rows = []

# Iterate over each row in the DataFrame
for index, row in summaries_war.iterrows():
    persona_dict = ast.literal_eval(row['Persona'])
    for persona, actions in persona_dict.items():
        # Copy the row and replace the 'character_actions' with the current character's data
        new_row = row.drop('Persona')  # Remove the old 'character_actions' column
        persona_series = pd.Series({
            'persona': persona,      # Add the persona name
            'actions': actions       # Add the actions
        })
        new_row = pd.concat([new_row, persona_series])
        new_rows.append(new_row)

# Convert the list of new rows into a new DataFrame
summaries_war2 = pd.DataFrame(new_rows)

summaries_war2['actions_clean'] = summaries_war2['actions'].map(lambda x: re.sub(r"[{}':,.!?]", '', ' '.join(x)))
summaries_war2["actions_clean"] = summaries_war2['actions_clean'].map(lambda x: x.lower())


# Display the new DataFrame
summaries_war2.head()

,name,release_date,revenue,runtime,languages,country,genres,plot,persona,actions,actions_clean
0,Army of Darkness,1992.0,21502796.0,81.0,English,United States of America,"Cult, Horror, Stop motion, Costume drama, Acti...","After being pulled through a time portal, Ash ...",Williams,{lands},lands
1,Army of Darkness,1992.0,21502796.0,81.0,English,United States of America,"Cult, Horror, Stop motion, Costume drama, Acti...","After being pulled through a time portal, Ash ...",Arthur,{war},war
2,Army of Darkness,1992.0,21502796.0,81.0,English,United States of America,"Cult, Horror, Stop motion, Costume drama, Acti...","After being pulled through a time portal, Ash ...",Ash Williams,"{grows, ducks, regains, king, arrives, capture...",grows ducks regains king arrives captured grab...
3,Army of Darkness,1992.0,21502796.0,81.0,English,United States of America,"Cult, Horror, Stop motion, Costume drama, Acti...","After being pulled through a time portal, Ash ...",Ash,"{kills, return, leads, saving, bringing, store...",kills return leads saving bringing store burie...
4,The Birth of a Nation,1915.0,50000000.0,190.0,"Silent film, English",United States of America,"Silent film, Indie, Costume drama, Epic, Black...",The film follows two juxtaposed families: the...,Ben Cameron,"{make, idolizes, leading, guilty, rescue, find...",make idolizes leading guilty rescue finds pers...


In [43]:
summaries_war2 = summaries_war2[summaries_war2['actions_clean'].str.split().str.len() > 1]
summaries_war2.head()

,name,release_date,revenue,runtime,languages,country,genres,plot,persona,actions,actions_clean
2,Army of Darkness,1992.0,21502796.0,81.0,English,United States of America,"Cult, Horror, Stop motion, Costume drama, Acti...","After being pulled through a time portal, Ash ...",Ash Williams,"{grows, ducks, regains, king, arrives, capture...",grows ducks regains king arrives captured grab...
3,Army of Darkness,1992.0,21502796.0,81.0,English,United States of America,"Cult, Horror, Stop motion, Costume drama, Acti...","After being pulled through a time portal, Ash ...",Ash,"{kills, return, leads, saving, bringing, store...",kills return leads saving bringing store burie...
4,The Birth of a Nation,1915.0,50000000.0,190.0,"Silent film, English",United States of America,"Silent film, Indie, Costume drama, Epic, Black...",The film follows two juxtaposed families: the...,Ben Cameron,"{make, idolizes, leading, guilty, rescue, find...",make idolizes leading guilty rescue finds pers...
5,The Birth of a Nation,1915.0,50000000.0,190.0,"Silent film, English",United States of America,"Silent film, Indie, Costume drama, Epic, Black...",The film follows two juxtaposed families: the...,Stoneman,"{younger, leaves, go}",younger leaves go
6,The Birth of a Nation,1915.0,50000000.0,190.0,"Silent film, English",United States of America,"Silent film, Indie, Costume drama, Epic, Black...",The film follows two juxtaposed families: the...,Elsie Stoneman,"{leave, gains, flees, faints, goes, does, meet...",leave gains flees faints goes does meets break...


In [44]:
documents_list= summaries_war2['actions_clean'].tolist()


In [ ]:
# print(documents_list)

words = [word for string in documents_list for word in string.split()]
# print(words[2])

word_counts = Counter(words)

unique_words = [word for word, count in word_counts.items() if count == 1]

print(unique_words)

updated_documents = [
    ' '.join([word for word in string.split() if word not in unique_words])
    for string in documents_list
]

print(updated_documents)

['slayer', 'hypnotises', 'departing', 'reinvents', 'interests', 'rehydrates', 'catwoman', 'focusing', 'teaming', 'fries', 'surpassed', 'recoils', 'ling', 'impede', 'jolting', 'bumbling', 'lick', 'impersonated', 'fide', 'foretells', 'pipes', 'endorses', 'undesirable', 'lobs', 'cybernetic', 'relived', 'ra', 'classified', 'outlawed', 'enslaved', 'overpowered', 'tore', 'defuse', 'consulting', 'goodspeed', 'absolves', 'reese', 'feuding', 'hannassey', 'resisting', 'pki', 'westley', 'imprison', 'judged', 'castigates', 'clownish', 'sustain', 'unloaded', 'stanton', 'clamber', 'instance', 'voted', 'u-571', 'surveys', 'sweetie', 'cleaner', 'shopping', 'scornful', 'aviator', 'quarterback', 'disperses', 'miscarried', 'lorene', 'galovitch', 'alma', 'yell', 'simulate', 'dejected', 'gasim', 'disciplined', 'tenuous', 'chuck', 'thinner', 'debilitating', 'linking', 'transfixed', 'reveling', 'tomb', 'abide', 'jedi', 'bukater', 'forbid', 'arrangement', 're-outfit', 'disarming', 'quaid', 'sweating', 'hypnot

In [31]:
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = CountVectorizer()  # Using default tokenizer (which respects token_pattern)

# Vectorize document using TF-IDF
tfidf = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        ngram_range = (1,1),
                        tokenizer = tokenizer.tokenize)

# Fit and Transform the documents
train_data = tfidf.fit_transform(documents_list)   

/home/zoe/anaconda3/envs/ada/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [32]:
# Define the number of topics or components
num_components=10

# Create LDA object
model=LatentDirichletAllocation(n_components=num_components)

# Fit and Transform SVD model on data
lda_matrix = model.fit_transform(train_data)

# Get Components 
lda_components=model.components_

In [33]:
# Print the topics with their terms
terms = tfidf.get_feature_names_out()

for index, component in enumerate(lda_components):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:7]
    top_terms_list=list(dict(top_terms_key).keys())
    print("Topic "+str(index)+": ",top_terms_list)  

Topic 0:  ['joins', 'try', 'reveals', 'takes', 'leads', 'uses', 'marries']
Topic 1:  ['finds', 'gives', 'sees', 'meets', 'tells', 'goes', 'learns']
Topic 2:  ['married', 'called', 'takes', 'tells', 'threatens', 'learn', 'finds']
Topic 3:  ['named', 'falls', 'wants', 'meet', 'tries', 'make', 'young']
Topic 4:  ['tells', 'finds', 'goes', 'tries', 'gets', 'refuses', 'comes']
Topic 5:  ['leaves', 'major', 'dead', 'takes', 'tries', 'love', 'revealed']
Topic 6:  ['shoots', 'takes', 'tells', 'return', 'kills', 'runs', 'commits']
Topic 7:  ['set', 'alive', 'come', 'arrive', 'work', 'finds', 'end']
Topic 8:  ['kills', 'appears', 'takes', 'escape', 'able', 'finds', 'tells']
Topic 9:  ['attempts', 'informs', 'sends', 'arrives', 'decides', 'holds', 'tells']
